<a href="https://colab.research.google.com/github/j0hndufek/Brain-Tumor-MRI-CNN/blob/main/BrainTumorMRI_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import preprocessing

import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
#from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [3]:
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train_img = []
train_labels = []

test_img = []
test_labels = []

#path_train = ('/Users/johndufek/Downloads/archive (2)/Training/')# code for local directory (e.g. in jupyter)
#path_test = ('/Users/johndufek/Downloads/archive (2)/Testing/')
path_train = ('/content/gdrive/MyDrive/archive (2)/Training/')
path_test = ('/content/gdrive/MyDrive/archive (2)/Testing/')

img_size= 300

for i in os.listdir(path_train):
    for j in os.listdir(path_train+i):
        train_img.append (cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size)))
        train_labels.append(i)

for i in os.listdir(path_test):
    for j in os.listdir(path_test+i):
        test_img.append (cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size)))
        test_labels.append(i)

train_img = (np.array(train_img))
test_img = (np.array(test_img))


train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(train_labels)]
test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]



In [6]:
# Transform function to normalize photo data
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [7]:
# Create Training and Validation data sets
train_x, val_x, train_y, val_y = train_test_split(np.array(train_img), np.array(train_labels), test_size = 0.20)

In [8]:
train_load=DataLoader(
    torchvision.datasets.ImageFolder(path_train,transform=transform),
    batch_size=64, shuffle=True
)
test_load=DataLoader(
    torchvision.datasets.ImageFolder(path_test,transform=transform),
    batch_size=32, shuffle=True
)

In [11]:
dataiter = iter(train_load)
images,labels = next(dataiter)

In [12]:
conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
pool = nn.MaxPool2d(2, 2)

In [13]:
x = conv1(images)
x = pool(x)
x = conv2(x)
x = pool(x)
x = conv3(x)
x = pool(x)
print(x.shape)

torch.Size([64, 128, 18, 18])


In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128*18*18, 512)
        self.fc2 = nn.Linear(512, 4)  # Output classes
        self.dropout = nn.Dropout(0.5)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128*18*18)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Configure Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
epochs = 30
lrn_rate = 0.001

# Initialize model, loss function, and optimizer
model = CNN().to(device)
loss_f = nn.CrossEntropyLoss()
opt = Adam(model.parameters(), lr=lrn_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_load:
        images, labels = images.to(device), labels.to(device)

        opt.zero_grad()

        outputs = model(images)
        loss = loss_f(outputs, labels)
        loss.backward()
        opt.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_load)
    train_accuracy = 100. * correct / total

    model.eval()
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_load:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_accuracy = 100. * test_correct / test_total

    print(f'Epoch [{epoch+1}/{epochs}] | '
          f'Train Loss: {train_loss:.4f} | '
          f'Train Acc: {train_accuracy:.2f}% | '
          f'Test Acc: {test_accuracy:.2f}%')

Epoch [1/30] | Train Loss: 3.7679 | Train Acc: 52.94% | Test Acc: 30.46%
Epoch [2/30] | Train Loss: 0.7199 | Train Acc: 68.84% | Test Acc: 36.04%
Epoch [3/30] | Train Loss: 0.6770 | Train Acc: 69.82% | Test Acc: 44.42%
Epoch [4/30] | Train Loss: 0.5576 | Train Acc: 76.55% | Test Acc: 50.00%
Epoch [5/30] | Train Loss: 0.5030 | Train Acc: 79.26% | Test Acc: 54.57%
Epoch [6/30] | Train Loss: 0.4685 | Train Acc: 80.62% | Test Acc: 51.52%
Epoch [7/30] | Train Loss: 0.4174 | Train Acc: 83.40% | Test Acc: 57.11%
Epoch [8/30] | Train Loss: 0.3799 | Train Acc: 84.13% | Test Acc: 58.12%
Epoch [9/30] | Train Loss: 0.3261 | Train Acc: 87.53% | Test Acc: 59.39%
Epoch [10/30] | Train Loss: 0.2881 | Train Acc: 88.68% | Test Acc: 61.42%
Epoch [11/30] | Train Loss: 0.2679 | Train Acc: 89.51% | Test Acc: 64.47%
Epoch [12/30] | Train Loss: 0.2187 | Train Acc: 91.91% | Test Acc: 63.45%
Epoch [13/30] | Train Loss: 0.2108 | Train Acc: 91.18% | Test Acc: 65.99%
Epoch [14/30] | Train Loss: 0.1952 | Train Acc: